### Data Set: 
https://github.com/mhjabreel/CharCnn_Keras/tree/master/data/ag_news_csv

The AG's news topic classification dataset is constructed by choosing the 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600. 

The classes are: 

* World
* Sports
* Business
* Science/Technology

#### For more information on how to use Lbl2Vec, visit the [API Guide](https://lbl2vec.readthedocs.io/en/latest/api.html#)

In [1]:
from lbl2vec import Lbl2Vec
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.parsing.preprocessing import strip_tags
from sklearn.metrics import f1_score

### Load data

In [3]:
# load train data
ag_train = pd.read_csv('data/train.csv',sep=',',header=None, names=['class','title','description'])

# load test data
ag_test = pd.read_csv('data/test.csv',sep=',',header=None, names=['class','title','description'])

# load labels with keywords
labels = pd.read_csv('data/labels.csv',sep=';')

# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

In [4]:
labels

,class_index,class_name,keywords,number_of_keywords
0,1,World,"[election, state, president, police, politics,...",11
1,2,Sports,"[olympic, football, sport, league, baseball, r...",32
2,3,Business,"[company, market, oil, consumers, exchange, bu...",10
3,4,Science/Technology,"[laboratory, computers, science, technology, w...",18


### Tokenize data

In [5]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

In [6]:
# add data set type column
ag_train['data_set_type'] = 'train'
ag_test['data_set_type'] = 'test'

# concat train and test data
ag_full_corpus = pd.concat([ag_train,ag_test]).reset_index(drop=True)

In [7]:
# tokenize and tag documents combined title + description for Lbl2Vec training
ag_full_corpus['tagged_docs'] = ag_full_corpus.apply(lambda row: TaggedDocument(tokenize(row['title'] + '. ' + row['description']), [str(row.name)]), axis=1)

In [8]:
# add doc_key column
ag_full_corpus['doc_key'] = ag_full_corpus.index.astype(str)

In [9]:
# add class_name column
ag_full_corpus = ag_full_corpus.merge(labels, left_on='class', right_on='class_index', how='left').drop(['class', 'keywords'], axis=1)

In [10]:
ag_full_corpus.head()

,title,description,data_set_type,tagged_docs,doc_key,class_index,class_name,number_of_keywords
0,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",train,"([wall, st, bears, claw, back, into, the, blac...",0,3,Business,10
1,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,train,"([carlyle, looks, toward, commercial, aerospac...",1,3,Business,10
2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,train,"([oil, and, economy, cloud, stocks, outlook, r...",2,3,Business,10
3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,train,"([iraq, halts, oil, exports, from, main, south...",3,3,Business,10
4,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",train,"([oil, prices, soar, to, all, time, record, po...",4,3,Business,10


### Train Doc2Vec model

In [11]:
# in order to get optimal Lbl2Vec results the given Doc2Vec model 
# should be trained with the parameters “dbow_words=1” and “dm=0”.
doc2vec_model = Doc2Vec(documents=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='train'], dbow_words=1, dm=1)

# Train Lbl2Vec

Train a new Lbl2Vec model using a pretrained [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec) model:
* keywords_list : iterable list of lists with descriptive keywords for each topic.
* doc2vec_model : pretrained [gensim.models.doc2vec.Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec) model. Lbl2Vec uses its word and document vectors to compute the label vectors.
* label_names : iterable list of custom names for each label. Label names and keywords of the same topic must have the same index.
* similarity_threshold : only documents with a higher similarity to the respective description keywords than this treshold are used to calculate the label embedding.
* min_num_docs : minimum number of documents that are used to calculate the label embedding. 
* epochs : number of iterations over the corpus.

In [12]:
# init model with parameters
lbl2vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), doc2vec_model=doc2vec_model, label_names=list(labels['class_name']), similarity_threshold=0.30, min_num_docs=100, epochs=10)

In [13]:
# train model
lbl2vec_model.fit()

2022-11-13 17:41:01,232 - Lbl2Vec - INFO - Load document and word embeddings
2022-11-13 17:41:01,235 - Lbl2Vec - INFO - Train label embeddings


# Predict topics of documents used to train Doc2Vec

Compute similarity scores of learned document vectors from documents that were used to train the Doc2Vec model to each of the learned label vectors. The similarity scores consist of cosine similarities and therefore have a value range of [-1,1].

In [14]:
# predict similarity scores
model_docs_lbl_similarities = lbl2vec_model.predict_model_docs()

2022-11-13 17:41:03,963 - Lbl2Vec - INFO - Get document embeddings from model
2022-11-13 17:41:04,061 - Lbl2Vec - INFO - Calculate document<->label similarities


In [15]:
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,World,Sports,Business,Science/Technology
0,0,Science/Technology,0.608025,0.469767,0.545740,0.392081,0.608025
1,1,Science/Technology,0.592419,0.408745,0.432462,0.413554,0.592419
2,2,Business,0.433717,0.275496,0.368149,0.433717,0.414448
3,3,World,0.495766,0.495766,0.157258,0.279696,0.248418
4,4,Science/Technology,0.495465,0.374365,0.398580,0.306961,0.495465


## Evaluate prediction of documents used to train Doc2Vec

In [16]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_train = model_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='train'], left_on='doc_key', right_on='doc_key')

In [17]:
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.6552833333333333


# Predict topics of unknown documents

Learn document vectors of new documents that were **not** used to train the Doc2Vec model and compute the similarity scores to each of the learned label vectors. The similarity scores consist of cosine similarities and therefore have a value range of [-1,1].

In [18]:
# predict similarity scores of new test documents (they were not used during Doc2Vec training)
new_docs_lbl_similarities = lbl2vec_model.predict_new_docs(tagged_docs=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='test'])

2022-11-13 17:58:45,903 - Lbl2Vec - INFO - Calculate document embeddings
2022-11-13 17:58:54,961 - Lbl2Vec - INFO - Calculate document<->label similarities


In [19]:
new_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,World,Sports,Business,Science/Technology
0,120000,Business,0.304743,0.194844,0.154270,0.304743,0.151537
1,120001,Sports,0.502020,0.344462,0.502020,0.233940,0.425520
2,120002,Science/Technology,0.391606,0.201942,0.191337,0.221430,0.391606
3,120003,Science/Technology,0.276807,0.208226,0.192992,0.203607,0.276807
4,120004,Science/Technology,0.211827,0.209541,0.086736,0.178669,0.211827


## Evaluate prediction of new documents

In [20]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_test = new_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')

In [21]:
y_true_test = evaluation_test['class_name']
y_pred_test = evaluation_test['most_similar_label']
print('F1 score:',f1_score(y_true_test, y_pred_test, average='micro'))

F1 score: 0.6360526315789473
